# Explanation

Here I create and query a SQLite database using python's standard API sqlite3. This is just a basic demonstration of my proficiency in SQLite.

The fictional tables we will be using are:

### ModelPrice

|Model ID| Price (USD)|
|---|---:|
|00001|8000.00|
|00002|8000.00|
|00003|15000.00|
|00004|3000.00|

### ModelReliability

|Model ID| # of Faults|
|---|:---:|
|00001|23|
|00002|8|
|00003|14|
|00004|11|

### ManufacturedProducts

|Product ID| Model ID|Date Manufactured|
|---|---|---|
|00001|00001|05/11/2020|
|00002|00001|05/13/2020|
|00003|00004|05/16/2020|
|00004|00001|05/16/2020|
|00005|00002|06/04/2020|
|00006|00002|06/08/2020|
|00007|00004|06/23/2020|
|00008|00003|06/29/2020|
|00009|00002|07/02/2020|
|00010|00002|07/09/2020|
|00011|00001|07/11/2020|
|00012|00003|07/14/2020|


### CustomerOwnership

|Product ID| Customer Name|
|---|---|
|00001|Amazon|
|00002|Amazon|
|00003|Amazon|
|00004|Boeing|
|00005|Boeing|
|00006|Walmart|
|00007|Boeing|
|00008|Walmart|
|00009|Amazon|
|00010|Boeing|





# Creation of the Database

In [53]:
import sqlite3 as sql
import pandas as pd

In [31]:
connection = sql.connect('example.db')
cur = connection.cursor()

#Make Tables
cur.execute('CREATE TABLE ModelPrice (ModelID INTEGER, Price REAL)')
cur.execute('CREATE TABLE ModelReliability (ModelID INTEGER, NumFaults INTEGER)')
cur.execute('CREATE TABLE ManufacturedProducts (ProductID INTEGER, ModelID Integer, DateManufact TEXT)')
cur.execute('CREATE TABLE CustomerOwnership (ProductID INTEGER, CompanyName Text)')

In [32]:
#Table Rows
data_1 = [(1,8000.),(2,8000.),(3,15000.),(4,3000.)]
data_2 = [(1,23),(2,8),(3,14),(4,11)]
data_3 = [(1,1,'05/11/2020'),(2,1,'05/13/2020'),(3,4,'05/16/2020'),
          (4,1,'05/16/2020'),(5,2,'06/04/2020'),(6,2,'06/08/2020'),
          (7,4,'06/23/2020'),(8,3,'06/29/2020'),(9,2,'07/02/2020'),
          (10,2,'07/09/2020'),(11,1,'07/11/2020'),(12,3,'07/14/2020')]
data_4 = [(1,'Amazon'),(2,'Amazon'),(3,'Amazon'),(4,'Boeing'),(5,'Boeing'),
          (6,'Walmart'),(7,'Boeing'),(8,'Walmart'),(9,'Amazon'),(10,'Boeing')]

#Insert Data Into Tables
cur.executemany('INSERT INTO ModelPrice VALUES(?,?)',data_1)
cur.executemany('INSERT INTO ModelReliability VALUES(?,?)',data_2)
cur.executemany('INSERT INTO ManufacturedProducts VALUES(?,?,?)',data_3)
cur.executemany('INSERT INTO CustomerOwnership VALUES(?,?)',data_4)

connection.commit()

connection.close()

# Question $\rightarrow$ Query $\rightarrow$ Answer

In [33]:
#Access Database and Create Backup... Just in Case
connection = sql.connect('example.db')
backup = sql.connect('backup.db')
connection.backup(backup)
backup.close()

cur = connection.cursor()

### Question #1: Which models of our products does each customer use?

In [52]:
df = pd.DataFrame(cur.execute('SELECT DISTINCT CompanyName, ModelID '
                              'FROM ManufacturedProducts AS MP INNER JOIN CustomerOwnership AS CO '
                              'ON MP.ProductID = CO.ProductID ').fetchall(),columns=['Name','ID'])
df.groupby('Name').ID.unique()

Name
Amazon     [1, 4, 2]
Boeing     [1, 2, 4]
Walmart       [2, 3]
Name: ID, dtype: object

### Question #2: What is our revenue by month?

In [78]:
#Revenue Only Comes From Products That Are Sold
df = pd.DataFrame(cur.execute('SELECT MAP.DateManufact, MOP.Price '
                              'FROM CustomerOwnership AS CO ' 
                              'INNER JOIN ManufacturedProducts AS MAP, ModelPrice AS MOP '
                              'ON CO.ProductID = MAP.ProductID AND MAP.ModelID = MOP.ModelID').fetchall(),
                 columns = ['DateManufact','Price'])
df['DateManufact'] = pd.to_datetime(df['DateManufact'])
df.groupby(df.DateManufact.dt.month).Price.sum()

DateManufact
5    27000.0
6    34000.0
7    16000.0
Name: Price, dtype: float64

### Question #3: How reliable is each model (weighted by cumulative days after manufacture date)?

In [77]:
df = pd.DataFrame(cur.execute('SELECT MP.DateManufact, MP.ModelID, MR.NumFaults '
                              'FROM ManufacturedProducts AS MP ' 
                              'INNER JOIN ModelReliability AS MR '
                              'ON MP.ModelID = MR.ModelID').fetchall(), columns = ['Date','ModelID','NumFaults'])

df['Days'] = (pd.to_datetime('today')-pd.to_datetime(df['Date'])).dt.days
grouped = df.groupby('ModelID')
print('Faults per day since production\n', grouped.NumFaults.mean()/grouped.Days.sum())

Faults per day since production
 ModelID
1    0.095833
2    0.055172
3    0.341463
4    0.103774
dtype: float64


### Question #4: How much revenue has come from each customer every month?

In [81]:
df = pd.DataFrame(cur.execute('SELECT CO.CompanyName, MAP.DateManufact, MOP.Price '
                              'FROM CustomerOwnership AS CO ' 
                              'INNER JOIN ManufacturedProducts AS MAP, ModelPrice AS MOP '
                              'ON CO.ProductID = MAP.ProductID AND MAP.ModelID = MOP.ModelID').fetchall(),
                 columns = ['Company','Month','Price'])
df['Month'] = pd.to_datetime(df['Month'])
df.groupby([df.Company,df.Month.dt.month]).Price.sum()

Company  Month
Amazon   5        19000.0
         7         8000.0
Boeing   5         8000.0
         6        11000.0
         7         8000.0
Walmart  6        23000.0
Name: Price, dtype: float64

In [82]:
connection.close()